In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Dependencies

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00


In [3]:
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW

import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm
import io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [4]:
data = pd.read_csv('/content/drive/MyDrive/NLP/data')

In [5]:
data.head(10)

,text,label
0,كشف جيسوس نافاس جناح مانشستر سيتي الجديد تقدم ...,Sports
1,دبي حمدي سعدتعتبر التطورات والمبادرات تقودها د...,Tech
2,تعاقد نادي سسكا موسكو الروسي المهاجم الدولي ال...,Sports
3,اشاد ادجاردو باوزا مدرب الارجنتين بنجم المنتخب...,Sports
4,توفي امس البائع المغربي المتجول نور الدين عدنا...,Politics
5,حوارجيهان شعيباذا للبيئه اصدقاء كثر يسعون لحما...,Religion
6,ابوظبي اسعد سيطرت اجواء الحذر والترقب اسواق ال...,Finance
7,يدهشك الكاتب والقاص عبدالرضا السجواني يتناول ذ...,Culture
8,يتمتع الكثير الاجهزه الرقميه صغيره الحجم وقتنا...,Tech
9,زوج يعرف الوفاء قدمت لزوجي معروف بدايه حياته ا...,Religion


# Encode the labels

In [6]:
data['encoded_label'] = data['label'].astype('category').cat.codes
data.head(20)

,text,label,encoded_label
0,كشف جيسوس نافاس جناح مانشستر سيتي الجديد تقدم ...,Sports,5
1,دبي حمدي سعدتعتبر التطورات والمبادرات تقودها د...,Tech,6
2,تعاقد نادي سسكا موسكو الروسي المهاجم الدولي ال...,Sports,5
3,اشاد ادجاردو باوزا مدرب الارجنتين بنجم المنتخب...,Sports,5
4,توفي امس البائع المغربي المتجول نور الدين عدنا...,Politics,3
5,حوارجيهان شعيباذا للبيئه اصدقاء كثر يسعون لحما...,Religion,4
6,ابوظبي اسعد سيطرت اجواء الحذر والترقب اسواق ال...,Finance,1
7,يدهشك الكاتب والقاص عبدالرضا السجواني يتناول ذ...,Culture,0
8,يتمتع الكثير الاجهزه الرقميه صغيره الحجم وقتنا...,Tech,6
9,زوج يعرف الوفاء قدمت لزوجي معروف بدايه حياته ا...,Religion,4


In [6]:
class_mapping = {}

# Manually add entries to the mapping dictionary
class_mapping[0] = "Culture"
class_mapping[1] = "Finance"
class_mapping[2] = "Medical"
class_mapping[3] = "Politics"
class_mapping[4] = "Religion"
class_mapping[5] = "Sports"
class_mapping[6] = "Tech"

for code, class_label in class_mapping.items():
    print(f"Code: {code} - Class: {class_label}")


Code: 0 - Class: Culture
Code: 1 - Class: Finance
Code: 2 - Class: Medical
Code: 3 - Class: Politics
Code: 4 - Class: Religion
Code: 5 - Class: Sports
Code: 6 - Class: Tech


# Import MARBERT

In [8]:
# Set the device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 7  # Number of classes in the SANAD dataset
pretrained_model_name = "aubmindlab/bert-base-arabertv2"  # Pretrained MARBERT model
max_seq_length = 128  # Maximum sequence length for input text
batch_size = 8
num_epochs = 8
learning_rate = 2e-5

# Load the tokenizer and the pretrained model
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=num_classes)
model.to(device)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Split the data

In [9]:
train_df, val_test_df = train_test_split(data, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

In [10]:
train_texts = train_df['text'].tolist()
train_labels = train_df['encoded_label'].tolist()
val_texts = val_df['text'].tolist()
val_labels = val_df['encoded_label'].tolist()

# Encode the training and validation datasets

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_seq_length)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_seq_length)


In [ ]:
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels)
)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
# Define the optimizer and the loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



# Fine-tune MARBERT

In [ ]:
# Fine-tuning loop
for epoch in range(num_epochs): 

    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)

    # Training
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Training Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

Epoch 1/8
----------
Training Loss: 0.1679
Epoch 2/8
----------
Training Loss: 0.0702
Epoch 3/8
----------
Training Loss: 0.0437
Epoch 4/8
----------
Training Loss: 0.0325
Epoch 5/8
----------
Training Loss: 0.0235
Epoch 6/8
----------
Training Loss: 0.0187
Epoch 7/8
----------
Training Loss: 0.0179
Epoch 8/8
----------
Training Loss: 0.0153


# Save model & tokenizer

In [11]:
model.save_pretrained('/content/model/model')
tokenizer.save_pretrained('/content/model/tokenizer')

('/content/model/tokenizer/tokenizer_config.json',
 '/content/model/tokenizer/special_tokens_map.json',
 '/content/model/tokenizer/vocab.txt',
 '/content/model/tokenizer/added_tokens.json')

# Load model & tokenizer

In [17]:
tokenizer = BertTokenizer.from_pretrained('/content/model/tokenizer')
model = BertForSequenceClassification.from_pretrained('/content/model/model')

# Perform some predictions

In [18]:
def predict_class(input_text):
  input_encoding = tokenizer(input_text, truncation=True, padding=True, max_length=max_seq_length)

  # Create input tensors
  input_ids = torch.tensor(input_encoding['input_ids']).unsqueeze(0).to(device)
  attention_mask = torch.tensor(input_encoding['attention_mask']).unsqueeze(0).to(device)

  # Make the prediction
  model.to(device)
  model.eval()
  with torch.no_grad():
      inputs = {
          'input_ids': input_ids,
          'attention_mask': attention_mask
      }
      outputs = model(**inputs)
      logits = outputs.logits
      probabilities = torch.softmax(logits, dim=1)
      predicted_label = torch.argmax(probabilities, dim=1).item()
      return class_mapping.get(predicted_label)

In [50]:
print(test_texts[3515])
print(test_labels[3515])
print(predict_class(test_texts[3515]))

اللهم بلغتهذه اللازمه رددها النبي صلي الله وسلم اطول خطبه الاخيره خطبه الوداع وهي لازمه عظيمه الدلاله مقامها لانها لخصت حياه كامله الفاظ معدودات فما حياه النبي صلي الله وسلم كلها بعملها وقولها وحركتها وسكونها حياه تبليغ وبلاغ فاصله خاتمه ابلغ قوله السلام يجود بنفسه جلال ربي الرفيع بلغت ولصدق الدلاله نري السمه الغالبه اسلوب النبي صلي الله وسلم كلامه المحفوظ ايدينا سمه الابلاغ سمه اخري السمه الجامعه سمه غيرها لانها اصل شامل تفرق سمات بمثابه الفروع وكلام النبي صلي الله وسلم المحفوظ ايدينا معاهدات ورسائل كتبت حينها واما خطب وادعيه ووصايا واجوبه اسئله كتبت حينها وروعيت الدقه المضاهاه رواياتها جهد المستطاع والابلاغ السمه المشتركه افانين الكلام جميعا حتي جري مجري القصص مجري الاوامر المرؤوسين مجري الدعاء يلقنه المسلم ليدعو الله مثاله انظر مثلا قصه اصحاب الغار الثلاثه وتوسلهم بصالح الاعمال جاء مختار مسلم ثلاثه نفر يمشون اخذهم المطر فاووا غار جبل فانحطت فم غارهم صخره الجبل فانطبقت عليهم من كتاب عبقريه محمد لعباس محمود العقاد
4
Religion


In [54]:
text = "وصول فريقي لنهائي الكونفدرالية مفاجأة.. حديث خاص مع نصر الدين النابي المدير الفني لفريق يانج أفريكانز عن التأهل لنهائي الكونفدرالية.. ورؤيته لمواجهة الأهلي والوداد"
print(predict_class(text))

Sports


In [46]:
text = "نفاذ جميع تذاكر مباراة النصر والاتفاق"
print(predict_class(text))

Sports


In [57]:
text = "ليفربول بداية المباراة, ليفربول نهاية المباراة"
print(predict_class(text))

Sports


In [47]:
text = "الأسطورة كرستيانو رونالدو يلعب آخر مباراة له بـقميص ريال مدريد"
print(predict_class(text))

Sports


In [59]:
text = "ألف مبروك للمنتخب المغربي الشقيق، وللكرة العربية، هذا الإنجاز التاريخي بالتأهل لنصف نهائي كأس العالم 💪💪💪🇲🇦🇲🇦🇲🇦"
print(predict_class(text))

Sports


In [14]:
text = "كما أتوجه بتحية الفخر والإجلال، للرئيس الراحل أنور السادات، الذي جسد بقراري الحرب والسلم قيم مصر الراسخة للأبد، التي تؤمن بالعدل والأمن والسلام، كل عام وأنتم بخير. "
print(predict_class(text))

Politics


In [43]:
text = "عيد الشرطة يأتي في الخامس والعشرين من يناير كل عام وذلك لإحياء ذكرى استشهاد ٥٠ ضابطا وجنديا وإصابة ٨٠ من قوات الشرطة المصرية البواسل في مدينة الإسماعيلية وهم يدافعون عن ثغورهم من الاحتلال الانجليزي في ١٩٥٢. ربنا يرحم كل ضابط مات دفاعا عن وطنه ودينه وأهله وناسه 🖤🤍"
print(predict_class(text))

Politics


In [44]:
text = "امبارح المظاهرات كانت عنيفة جدا والشرطة استخدمت الغاز المسيل للدموع عشلن تفرق المتظاهرين"
print(predict_class(text))

Politics


In [45]:
text = "لجيش الصهيوني الارهابي يفجر منزل الشهيد معتز الخواجا في بلدة نعلين غرب رام الله. ضمن سياسة العقاب الجماعي التي ينتهجها تجاه الشعب الفلسطيني."
print(predict_class(text))

Politics


In [24]:
text = "احرص على أخذ لقاح ضد الإنفلونزا والتهاب الرئة والتطعيمات اللازمة للوقاية من الإصابة بمرض الربو🙏"
print(predict_class(text))

Medical


In [40]:
text = "انقذوا مستشفى 57357"
print(predict_class(text))

Medical


In [42]:
text = " نرحب بانضمام الدكتورة ميادة عفت حلمي أخصآئي التغذية والسمنة والنحافة"
print(predict_class(text))

Medical


In [26]:
text = "رمضان كريم ، يارب يكون شهر مليان خير عليكم حبايبي الله يجعلكم من صائمينه وقائمينه ومبارك عليكم الشهر"
print(predict_class(text))

Religion


In [36]:
text = "ليلة غُسلت فيها أحزان الحبيب  المصطفى بعد عام الحزن  اللهم كما جعلتها ليلة دخول الفرح والسرور على قلبه الشريف بعد أن طال حزنه فَاجعلها ليلة فرح وسرور علينا وعلى أمة سيدنا محمد صلى الله عليه وسلم"
print(predict_class(text))

Religion


In [37]:
text = "مفتي عام المملكة: عناية ورعاية الدولة -أيدها الله- بخدمة الحرمين الشريفين، جعل منهما منارة معمارية حسية ومنارة دينية معنوية."
print(predict_class(text))

Religion


In [28]:
text = "صعود أسعار #النفط يزيد جاذبية التقنيات الخضراء، برغم ذلك ليس من السهل إجراء التحول على الفور وبسهولة مطلقة"
print(predict_class(text))

Finance


In [29]:
text = "#الإمارات و #السعودية و #مصر و #المغرب و #قطر بين أفضل الأسواق الناشئة بمؤشر ثقة الاستثمار الأجنبي المباشر لعام 2023"
print(predict_class(text))

Finance


In [32]:
text = "كيف سيعمل دمج الذكاء الاصطناعي مع نتائج بحث جوجل؟"
print(predict_class(text))

Tech


In [33]:
text = "أجرت الصين تمرينًا كبيرًا على الاستجابة للزلازل.  استخدمت بكين التكنولوجيا والمعدات الحديثة.  لقطات حصرية من المشهد"
print(predict_class(text))

Tech


In [34]:
text = "تحويل الهاتف لكاميرا سيارة.. ميزة رائعة قادمة لـ أندرويد"
print(predict_class(text))

Tech


In [35]:
text = "عندما تستخدم تقنيات الأمن السيبراني فأنت تقوم بدعم موظفي تكنولوجيا المعلومات لديك في مواجهة التهديدات الرقمية المحتملة، وتوفر لهم التكنولوجيا الحديثة التي تلزمهم لحماية المنشأة"
print(predict_class(text))

Tech


In [49]:
text = "اخيراً روايه اانا مذنب متحمسه جددداً ل قرائتها وتشرفت جداً بشوفته"
print(predict_class(text))

Culture


In [50]:
text = " أعلنت لجنة تحكيم مهرجان جمعية الفيلم السنوى فى دورته 49 للسينما المصرية عن فوز فيلم كيرة والجن بتسع جوائز في الدورة الـ 49..."
print(predict_class(text))

Culture


In [52]:
text = "في معرض الشارقة الدولي للكتاب 🇦🇪 عرف الجمهور إزاي تحول كريم عبد العزيز إلى يحيي راشد"
print(predict_class(text))

Culture


In [54]:
text = "عرض خاص لفيلم تراب الماس في دولة ألبانيا 🇦🇱 ضمن الأسبوع الثقافي المصري...  @jkn_al"
print(predict_class(text))

Culture


# Evaluate the model

In [12]:
test_texts = test_df['text'].tolist()
test_labels = test_df['encoded_label'].tolist()

In [13]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_seq_length)


In [14]:
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels)
)

In [15]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model = model.to(device)
def test_model(model, dataloader):
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1)

            true_labels.extend(labels.tolist())
            predicted_labels.extend(predicted_class.tolist())

    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='macro')
    recall = recall_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')
    return accuracy, precision, recall, f1


In [37]:
test_accuracy, test_precision, test_recall, test_f1 = test_model(model, test_dataloader)


In [38]:
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Test Accuracy: 0.9820
Test Precision: 0.9821
Test Recall: 0.9822
Test F1 Score: 0.9821
